In [1]:
import os
import pymysql
import pandas as pd
password = os.environ.get('MYSQL_PASSWORD')

In [2]:
try:
    con = pymysql.connect(
                            host='localhost',
                            user='root',
                            password=password,
                            charset='utf8mb4'
                            )
    print("Connection to the database was successful!")
except pymysql.Error as e:
    print(f"An error occurred while connecting to the database: {e}")


Connection to the database was successful!


In [14]:
# calling this function makes life easy and just need to pass the name of the database I want to make a query on
# It serves like USE keyword in SQL

def connector (database, password=password, host='localhost', 
               user='root', charset='utf8mb4',):
    
    conn = pymysql.connect(
                        host=host,
                        user=user,
                        password=password,
                        charset=charset, 
                        database=database)
    
    return conn

In [4]:
# How to get the names of all the databases in my MySQL server 

# I can use the cursor object from my database connection to execute SQL commands and retrieve information 
# about my databases. Here's an example of how I can use the **SHOW DATABASES** SQL command to retrieve the 
# names of all databases in my MySQL server using my connector function:


# Create a database connection
conn = connector(database='sql_store')

# Create a cursor object
cursor = conn.cursor()

# Execute the SHOW DATABASES command
cursor.execute('SHOW DATABASES')

# Fetch all the results as a list of tuples
results = cursor.fetchall()

# Print the names of all databases in the MySQL server
for row in results:
    print(row[0])

# Close the cursor and connection
cursor.close()
conn.close()


information_schema
mysql
performance_schema
q1757
sql_hr
sql_inventory
sql_invoicing
sql_store
sys


In [5]:
def delete_database(database):
    conn = connector(database=None)
    cursor = conn.cursor()

    # Replace <database_name> with the name of the database you want to delete
    database_name = database

    # Execute the DROP DATABASE SQL command
    cursor.execute(f"DROP DATABASE {database_name}")

    # Commit the transaction
    conn.commit()

    # Close the connection
    conn.close()
    

In [ ]:
delete_database('q1757')

In [15]:
def database_creator(database_name):
    
    conn = connector(database=None)
    
    # Create a cursor object
    cursor = conn.cursor()

    # Execute the CREATE DATABASE SQL command
    cursor.execute(f"CREATE DATABASE {database_name}")

    # Commit the transaction
    conn.commit()

    # Close the connection
    conn.close()


In [16]:
def insert_data_to_table(database_name, table_name, schema, data):
    # Connect to the database
    conn = connector(database_name)

    # Create a cursor object
    cursor = conn.cursor()

    # Create the table with the given schema if it does not exist
    cursor.execute(f"CREATE TABLE IF NOT EXISTS {table_name} ({schema})")

    # Truncate the table to remove any existing data
    cursor.execute(f"TRUNCATE TABLE {table_name}")

    # Insert data into the table
    for row in data:
        placeholders = ",".join(["%s" for _ in range(len(row))])
        query = f"INSERT INTO {table_name} VALUES ({placeholders})"
        cursor.execute(query, row)

    # Commit the transaction to save the changes
    conn.commit()

    # Close the connection
    conn.close()


# Leetcode q1757:

In [ ]:
database_creator ('q1757')

In [ ]:
# here is the SQL Schema for the question 1757 on Leetcode - database 

Create table If Not Exists Products (product_id int, low_fats ENUM('Y', 'N'), recyclable ENUM('Y','N'))
Truncate table Products
insert into Products (product_id, low_fats, recyclable) values ('0', 'Y', 'N')
insert into Products (product_id, low_fats, recyclable) values ('1', 'Y', 'Y')
insert into Products (product_id, low_fats, recyclable) values ('2', 'N', 'Y')
insert into Products (product_id, low_fats, recyclable) values ('3', 'Y', 'Y')
insert into Products (product_id, low_fats, recyclable) values ('4', 'N', 'N')

In [9]:
schema = "product_id int, low_fats ENUM('Y', 'N'), recyclable ENUM('Y', 'N')"
data = [("0", "Y", "N"), ("1", "Y", "Y"), ("2", "N", "Y"), ("3", "Y", "Y"), ("4", "N", "N")]

insert_data_to_table("q1757", "Products", schema, data)


In [10]:
# to get my tables' names in the datanase

query = "SELECT table_name FROM information_schema.tables WHERE table_schema='q1757';"

# Execute the query and store the results in a Pandas DataFrame
tables = pd.read_sql_query(query, con)

# Print the list of tables
print(tables)


  TABLE_NAME
0   Products


In [11]:
con = connector('q1757')

In [12]:
pd.read_sql("SELECT *\
            FROM Products p", con)

,product_id,low_fats,recyclable
0,0,Y,N
1,1,Y,Y
2,2,N,Y
3,3,Y,Y
4,4,N,N


In [13]:
pd.read_sql("SELECT p.product_id\
            FROM Products p\
            WHERE low_fats = 'Y' AND recyclable = 'Y'", con)

,product_id
0,1
1,3


# Leetcode q1378:

In [ ]:
Create table If Not Exists Employees (id int, name varchar(20))
Create table If Not Exists EmployeeUNI (id int, unique_id int)
Truncate table Employees
insert into Employees (id, name) values ('1', 'Alice')
insert into Employees (id, name) values ('7', 'Bob')
insert into Employees (id, name) values ('11', 'Meir')
insert into Employees (id, name) values ('90', 'Winston')
insert into Employees (id, name) values ('3', 'Jonathan')
Truncate table EmployeeUNI
insert into EmployeeUNI (id, unique_id) values ('3', '1')
insert into EmployeeUNI (id, unique_id) values ('11', '2')
insert into EmployeeUNI (id, unique_id) values ('90', '3')

In [17]:
database_creator('q1378')

In [20]:
schema = "id int, name varchar(20)"
data = [('1', 'Alice'), ('7', 'Bob'), ('11', 'Meir'), ('90', 'Winston'), ('3', 'Jonathan')]

insert_data_to_table("q1378", "Employees", schema, data)

In [24]:
schema = "id int, unique_id int"
data = [('3', '1'), ('11', '2'), ('90', '3')]

insert_data_to_table("q1378", "EmployeeUNI", schema, data)

In [27]:
# to get my tables' names in the datanase

query = "SELECT table_name FROM information_schema.tables WHERE table_schema='q1378';"

# Execute the query and store the results in a Pandas DataFrame
tables = pd.read_sql_query(query, con)

# Print the list of tables
print(tables)


    TABLE_NAME
0  EmployeeUNI
1    Employees


In [26]:
con = connector('q1378')
pd.read_sql("SELECT *\
            FROM Employees", con)

,id,name
0,1,Alice
1,7,Bob
2,11,Meir
3,90,Winston
4,3,Jonathan


In [29]:
con = connector('q1378')
pd.read_sql("SELECT *\
            FROM EmployeeUNI", con)

,id,unique_id
0,3,1
1,11,2
2,90,3


In [ ]:
# here